In [ ]:
import os 
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
import matplotlib as mpl 
# import missingno as msno
import datetime as dt
import seaborn as sns

from src import config

In [ ]:
customers = pd.read_csv(config.RAW_FILE_PATH / 'olist_customers_dataset.csv')
orders = pd.read_csv(config.RAW_FILE_PATH / 'olist_orders_dataset.csv')
order_items = pd.read_csv(config.RAW_FILE_PATH / 'olist_order_items_dataset.csv')

In [ ]:
order_items.info()
customers.info()
orders.head()

In [ ]:
rfm_order_items = order_items.groupby('order_id').agg({'price': sum, 'freight_value': sum, 'shipping_limit_date': max}).reset_index() 

In [ ]:
rfm_orders = orders.merge(rfm_order_items, on='order_id', how='inner')

In [ ]:
rfm = customers.merge(rfm_orders, on='customer_id', how='inner')

In [ ]:
rfm['order_status'].value_counts()

In [ ]:
rfm = rfm.loc[rfm['order_status'] == 'delivered']

In [ ]:
datetime_cols = [
  'order_purchase_timestamp',
  'order_approved_at',
  'order_delivered_carrier_date',
  'order_delivered_customer_date',
  'order_estimated_delivery_date',
]

In [ ]:
for col in datetime_cols: 
    rfm[col] = pd.to_datetime(rfm[col])

In [ ]:
rfm.info()

In [ ]:
rfm = rfm.dropna(axis=0)

In [ ]:
max_date = max(rfm['order_purchase_timestamp']) + dt.timedelta(days=1)

In [ ]:
rfm = rfm.groupby('customer_unique_id').agg({
    'order_purchase_timestamp': lambda x: (max_date - x.max()).days,
    'customer_id': 'count',
    'price': sum
}).reset_index()

In [ ]:
rfm.columns = ['customer_unique_id', 'recency', 'frequency', 'monetary']

In [ ]:
rfm.describe()

In [ ]:
rfm.to_csv(config.INT_FILE_PATH / 'rfm.csv')